# <a id='toc1_'></a>[Rusty Bargain ¿Cuánto vale mi carro?](#toc0_)

**Tabla de Contenido**<a id='toc0_'></a>    
- [Rusty Bargain ¿Cuánto vale mi carro?](#toc1_)    
  - [Descripción del proyecto](#toc1_1_)    
  - [Descripción de los datos](#toc1_2_)    
  - [Preparación de datos](#toc1_3_)    
    - [Imporación de librerías](#toc1_3_1_)    
    - [Carga de datos](#toc1_3_2_)    
    - [EDA](#toc1_3_3_)    
    - [Selección de columnas](#toc1_3_4_)    
    - [Formato a las columnas](#toc1_3_5_)    
    - [Duplicados explicitos](#toc1_3_6_)    
    - [Acotamiento de datos](#toc1_3_7_)    
    - [Valores ausentes](#toc1_3_8_)    
    - [Columnas con tipo de dato correcto](#toc1_3_9_)    
    - [Correlaciones](#toc1_3_10_)    
    - [Dividir el dataset en trenamiento y validación](#toc1_3_11_)    
    - [Escalado de datos](#toc1_3_12_)    
  - [Entrenamiento del modelo](#toc1_4_)    
    - [Modelo de regresión lineal](#toc1_4_1_)    
    - [Modelo de bosque aleatorio basico](#toc1_4_2_)    
    - [Catboost](#toc1_4_3_)    
    - [XGBoost](#toc1_4_4_)    
  - [Análisis del modelo](#toc1_5_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## <a id='toc1_1_'></a>[Descripción del proyecto](#toc0_)
El servicio de venta de autos usados Rusty Bargain está desarrollando una aplicación para atraer nuevos clientes. Gracias a esa app, puedes averiguar rápidamente el valor de mercado de tu coche. Tienes acceso al historial: especificaciones técnicas, versiones de equipamiento y precios. Tienes que crear un modelo que determine el valor de mercado.
A Rusty Bargain le interesa:
- La calidad de la predicción.
- La velocidad de la predicción.
- El tiempo requerido para el entrenamiento.

## <a id='toc1_2_'></a>[Descripción de los datos](#toc0_)
El dataset está almacenado en el archivo `../datasets/car_data.csv`.
Características:  
- DateCrawled — fecha en la que se descargó el perfil de la base de datos
- VehicleType — tipo de carrocería del vehículo
- RegistrationYear — año de matriculación del vehículo
- Gearbox — tipo de caja de cambios
- Power — potencia (CV)
- Model — modelo del vehículo
- Mileage — kilometraje (medido en km de acuerdo con las especificidades regionales del conjunto de datos)
- RegistrationMonth — mes de matriculación del vehículo
- FuelType — tipo de combustible
- Brand — marca del vehículo
- NotRepaired — vehículo con o sin reparación
- DateCreated — fecha de creación del perfil
- NumberOfPictures — número de fotos del vehículo
- PostalCode — código postal del propietario del perfil (usuario)
- LastSeen — fecha de la última vez que el usuario estuvo activo  

Objetivo:
- Price — precio (en euros)

## <a id='toc1_3_'></a>[Preparación de datos](#toc0_)

### <a id='toc1_3_1_'></a>[Imporación de librerías](#toc0_)

In [2]:
import lightgbm as lgb
import numpy as np
import pandas as pd
import xgboost as xgb
from catboost import CatBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

### <a id='toc1_3_2_'></a>[Carga de datos](#toc0_)

In [2]:
# Carga de datos
car_data = pd.read_csv('../datasets/car_data.csv', parse_dates=['DateCrawled','DateCreated','LastSeen']) 

### <a id='toc1_3_3_'></a>[EDA](#toc0_)

In [3]:
# Mostrar 10 filas al azar
car_data.sample(10)

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Mileage,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
233012,2016-01-04 05:36:00,2300,sedan,2003,manual,125,carisma,125000,5,petrol,mitsubishi,no,2016-01-04,0,80809,2016-05-04 03:17:00
326525,2016-08-03 16:06:00,350,small,1998,manual,54,corsa,150000,4,petrol,opel,no,2016-08-03,0,56412,2016-11-03 12:16:00
278161,2016-04-04 09:55:00,3850,convertible,1997,manual,0,z_reihe,150000,3,petrol,bmw,no,2016-04-04,0,40235,2016-06-04 11:17:00
304405,2016-12-03 14:06:00,550,small,2001,manual,75,corsa,150000,7,petrol,opel,yes,2016-12-03,0,56332,2016-03-17 13:45:00
139459,2016-11-03 12:46:00,999,sedan,1999,manual,90,a_klasse,150000,11,gasoline,mercedes_benz,yes,2016-11-03,0,75015,2016-07-04 06:17:00
143528,2016-09-03 10:37:00,7800,wagon,2006,auto,163,3er,150000,11,gasoline,bmw,no,2016-09-03,0,13507,2016-03-15 08:44:00
173194,2016-03-29 23:56:00,9125,sedan,2006,auto,272,other,125000,6,petrol,mercedes_benz,no,2016-03-29,0,14715,2016-06-04 21:18:00
168639,2016-03-13 14:47:00,650,small,1997,manual,50,polo,150000,3,petrol,volkswagen,NaN,2016-03-13,0,14532,2016-06-04 16:17:00
146562,2016-03-23 14:50:00,750,small,2001,manual,84,punto,150000,10,petrol,fiat,no,2016-03-23,0,85406,2016-03-27 19:44:00
9167,2016-03-16 20:43:00,1900,wagon,2002,manual,0,focus,125000,4,petrol,ford,no,2016-03-16,0,4357,2016-03-23 08:17:00


In [4]:
# Verificar valores ausentes, tipos de datos, número de observaciones
car_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   DateCrawled        354369 non-null  datetime64[ns]
 1   Price              354369 non-null  int64         
 2   VehicleType        316879 non-null  object        
 3   RegistrationYear   354369 non-null  int64         
 4   Gearbox            334536 non-null  object        
 5   Power              354369 non-null  int64         
 6   Model              334664 non-null  object        
 7   Mileage            354369 non-null  int64         
 8   RegistrationMonth  354369 non-null  int64         
 9   FuelType           321474 non-null  object        
 10  Brand              354369 non-null  object        
 11  NotRepaired        283215 non-null  object        
 12  DateCreated        354369 non-null  datetime64[ns]
 13  NumberOfPictures   354369 non-null  int64   

In [5]:
# Estadística descriptiva
car_data.describe()

,Price,RegistrationYear,Power,Mileage,RegistrationMonth,NumberOfPictures,PostalCode
count,354369.000000,354369.000000,354369.000000,354369.000000,354369.000000,354369.0,354369.000000
mean,4416.656776,2004.234448,110.094337,128211.172535,5.714645,0.0,50508.689087
std,4514.158514,90.227958,189.850405,37905.341530,3.726421,0.0,25783.096248
min,0.000000,1000.000000,0.000000,5000.000000,0.000000,0.0,1067.000000
25%,1050.000000,1999.000000,69.000000,125000.000000,3.000000,0.0,30165.000000
50%,2700.000000,2003.000000,105.000000,150000.000000,6.000000,0.0,49413.000000
75%,6400.000000,2008.000000,143.000000,150000.000000,9.000000,0.0,71083.000000
max,20000.000000,9999.000000,20000.000000,150000.000000,12.000000,0.0,99998.000000


**Nota:**
- Cambiar el formato del nombre de las columnas (minúsculas y snake case).
- Las columnas DateCrawled, DateCreated y LastSeen son fechas que no se relacionan con el precio, pueden ser eliminadas.
- Las columna NumberOfPictures tiene un mínimo y máximo de 0, puede ser eliminada.

**Nota 2:**  

Debido a que se tienen 354 mil datos, se ha tomado la desición de discriminar la información con valores dudosos.

- Price tiene valores de 0 a 20000. Se acotara el precio a un mínimo de 100.
- RegistrationYear tiene valores de 0 a 9999. Se acotara el año de 1900 a 2024.
- RegistrationMonth tiene valores de 0 a 12. Deberian ser unicamente 12 meses, se eliminaran los valores con el mes 0.

### <a id='toc1_3_4_'></a>[Selección de columnas](#toc0_)

In [6]:
# Eliminar columnas irrelevantes
car_data_cln = car_data.drop(['NumberOfPictures', 'DateCrawled', 'DateCreated', 'LastSeen'], axis=1)

### <a id='toc1_3_5_'></a>[Formato a las columnas](#toc0_)

In [7]:
# Dar formato al nombres de las columnas: minúsculas y snake_case
car_data_cln.columns = ['price', 'vehicle_type', 'registration_year', 'gearbox',
                        'power', 'model', 'mileage', 'registration_month', 'fuel_type', 'brand',
                        'not_repaired', 'postal_code']

### <a id='toc1_3_6_'></a>[Duplicados explicitos](#toc0_)

In [8]:
# Verificar filas totalmente duplicadas
car_data_cln[car_data_cln.duplicated()]

,price,vehicle_type,registration_year,gearbox,power,model,mileage,registration_month,fuel_type,brand,not_repaired,postal_code
3551,1670,sedan,1999,manual,75,golf,150000,8,petrol,volkswagen,no,52388
3786,2999,sedan,2002,manual,101,golf,150000,6,gasoline,volkswagen,no,32756
3907,500,small,1999,manual,55,corsa,150000,12,petrol,opel,no,51377
4134,18750,sedan,2014,manual,150,golf,20000,9,gasoline,volkswagen,no,38518
4222,14500,wagon,2007,manual,140,a6,100000,3,gasoline,audi,no,94060
...,...,...,...,...,...,...,...,...,...,...,...,...
354336,3299,NaN,2005,auto,0,outlander,150000,4,petrol,mitsubishi,NaN,17034
354337,11500,sedan,2004,auto,445,7er,125000,0,petrol,bmw,NaN,4107
354352,6500,sedan,2003,auto,145,e_klasse,150000,3,gasoline,mercedes_benz,no,60437
354355,4400,sedan,2008,manual,105,leon,150000,7,gasoline,seat,no,45896


In [9]:
# Verificar una observación al azar
car_data_cln[(car_data_cln['registration_year']==1999) & (car_data_cln['price']==1670)]

,price,vehicle_type,registration_year,gearbox,power,model,mileage,registration_month,fuel_type,brand,not_repaired,postal_code
824,1670,sedan,1999,manual,75,golf,150000,8,petrol,volkswagen,no,52388
3551,1670,sedan,1999,manual,75,golf,150000,8,petrol,volkswagen,no,52388
163866,1670,small,1999,manual,58,clio,80000,2,petrol,renault,no,56412
232731,1670,sedan,1999,manual,75,astra,150000,8,petrol,opel,no,23701
308127,1670,bus,1999,manual,115,zafira,150000,4,petrol,opel,no,71634
352652,1670,sedan,1999,manual,0,astra,150000,5,petrol,opel,NaN,24109


**Nota:**
Existen 21333 filas duplicadas. Contienen postal_code, el mismo mes, el mismo millaje... es demasiada coincidencia, vamos a eliminarlas.

In [10]:
# Eliminar filas totalmente duplicadas
car_data_cln.drop_duplicates(inplace=True)

### <a id='toc1_3_7_'></a>[Acotamiento de datos](#toc0_)

In [11]:
# Acotar los datos de las columnas regitration_year, power, price, registration_month
# Debido a datos que son dudosos

filtro_reg_year = (
    (car_data_cln['registration_year'] > 1900)
    & (car_data_cln['registration_year'] < 2024)
)

filtro_power = (
    (car_data_cln['power']>0)
)

filtro_price = (
    (car_data_cln['price'] > 100)
)

filtro_reg_month = (
    (car_data_cln['registration_month'] > 0)
)

car_data_cln = car_data_cln[filtro_reg_year & filtro_power & filtro_price & filtro_reg_month]
car_data_cln

,price,vehicle_type,registration_year,gearbox,power,model,mileage,registration_month,fuel_type,brand,not_repaired,postal_code
1,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,66954
2,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,90480
3,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,91074
4,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,60437
5,650,sedan,1995,manual,102,3er,150000,10,petrol,bmw,yes,33775
...,...,...,...,...,...,...,...,...,...,...,...,...
354361,5250,NaN,2016,auto,150,159,150000,12,NaN,alfa_romeo,no,51371
354362,3200,sedan,2004,manual,225,leon,150000,5,petrol,seat,yes,96465
354366,1199,convertible,2000,auto,101,fortwo,125000,3,petrol,smart,no,26135
354367,9200,bus,1996,manual,102,transporter,150000,3,gasoline,volkswagen,no,87439


**Nota:**  
Debido a que aun contamos con una gran cantidad de datos (267mil), se eliminaran aquellos con valores ausentes. 

### <a id='toc1_3_8_'></a>[Valores ausentes](#toc0_)

In [12]:
# Eliminar valores ausentes 
car_data_cln.dropna(inplace=True)
car_data_cln

,price,vehicle_type,registration_year,gearbox,power,model,mileage,registration_month,fuel_type,brand,not_repaired,postal_code
3,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,91074
4,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,60437
5,650,sedan,1995,manual,102,3er,150000,10,petrol,bmw,yes,33775
6,2200,convertible,2004,manual,109,2_reihe,150000,8,petrol,peugeot,no,67112
10,2000,sedan,2004,manual,105,3_reihe,150000,12,petrol,mazda,no,96224
...,...,...,...,...,...,...,...,...,...,...,...,...
354359,7900,sedan,2010,manual,140,golf,150000,7,gasoline,volkswagen,no,75223
354360,3999,wagon,2005,manual,3,3er,150000,5,gasoline,bmw,no,81825
354362,3200,sedan,2004,manual,225,leon,150000,5,petrol,seat,yes,96465
354366,1199,convertible,2000,auto,101,fortwo,125000,3,petrol,smart,no,26135


### <a id='toc1_3_9_'></a>[Columnas con tipo de dato correcto](#toc0_)

In [13]:
# Ver los valores del tipo de vehículos
car_data_cln.vehicle_type.value_counts()

sedan          62597
small          49146
wagon          43977
bus            20719
convertible    14253
coupe          10501
suv             8298
other           1325
Name: vehicle_type, dtype: int64

In [14]:
# Ver los valores de la caja de cambios
car_data_cln.gearbox.value_counts()

manual    166731
auto       44085
Name: gearbox, dtype: int64

In [15]:
# Ver los modelos
car_data_cln.model.value_counts()

golf                  17542
other                 15911
3er                   12819
polo                   7375
corsa                  6758
                      ...  
serie_3                   3
elefantino                3
samara                    2
range_rover_evoque        1
rangerover                1
Name: model, Length: 249, dtype: int64

In [16]:
# Ver los tipos de combustible
car_data_cln.fuel_type.value_counts()

petrol      138061
gasoline     68806
lpg           3302
cng            393
hybrid         161
other           47
electric        46
Name: fuel_type, dtype: int64

In [17]:
# Ver las marcas
car_data_cln.brand.value_counts().sort_index()

alfa_romeo        1457
audi             18869
bmw              23638
chevrolet         1097
chrysler           863
citroen           3195
dacia              635
daewoo             277
daihatsu           425
fiat              5312
ford             14336
honda             1706
hyundai           2408
jaguar             345
jeep               446
kia               1608
lada               116
lancia             260
land_rover         366
mazda             3437
mercedes_benz    20799
mini              2433
mitsubishi        1759
nissan            2960
opel             21767
peugeot           6665
porsche            489
renault           9559
rover              221
saab               361
seat              4254
skoda             3913
smart             2935
subaru             458
suzuki            1458
toyota            3232
trabant            137
volkswagen       44495
volvo             2125
Name: brand, dtype: int64

In [18]:
# Ver los valores de reparación
car_data_cln.not_repaired.value_counts()

no     189433
yes     21383
Name: not_repaired, dtype: int64

**Notas:**
- VehicleType, Gearbox, FuelType, de object a category.
- NotRepaired de objetct a bool. 

In [19]:
# Cambiar los tipos de datos de vehicle_type, fuel_type y gearbox
car_data_cln['vehicle_type'] = car_data_cln['vehicle_type'].astype('category')
car_data_cln['fuel_type'] = car_data_cln['fuel_type'].astype('category')
car_data_cln['gearbox'] = car_data_cln['gearbox'].astype('category')

# Convertir NotRepaired a bool
mapeo = {'yes': True, 'no': False}
car_data_cln['not_repaired'] = car_data_cln['not_repaired'].map(mapeo)

car_data_cln.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 210816 entries, 3 to 354367
Data columns (total 12 columns):
 #   Column              Non-Null Count   Dtype   
---  ------              --------------   -----   
 0   price               210816 non-null  int64   
 1   vehicle_type        210816 non-null  category
 2   registration_year   210816 non-null  int64   
 3   gearbox             210816 non-null  category
 4   power               210816 non-null  int64   
 5   model               210816 non-null  object  
 6   mileage             210816 non-null  int64   
 7   registration_month  210816 non-null  int64   
 8   fuel_type           210816 non-null  category
 9   brand               210816 non-null  object  
 10  not_repaired        210816 non-null  bool    
 11  postal_code         210816 non-null  int64   
dtypes: bool(1), category(3), int64(6), object(2)
memory usage: 15.3+ MB


### <a id='toc1_3_10_'></a>[Correlaciones](#toc0_)

In [20]:
# Correlaciones de valores númericos
car_data_cln.corr()

,price,registration_year,power,mileage,registration_month,not_repaired,postal_code
price,1.000000,0.556764,0.193507,-0.405617,0.008747,-0.207749,0.063686
registration_year,0.556764,1.000000,0.054869,-0.371042,-0.003397,-0.116393,0.034595
power,0.193507,0.054869,1.000000,0.047089,0.006609,-0.015003,0.015637
mileage,-0.405617,-0.371042,0.047089,1.000000,0.007451,0.087378,-0.015794
registration_month,0.008747,-0.003397,0.006609,0.007451,1.000000,-0.010660,-0.004568
not_repaired,-0.207749,-0.116393,-0.015003,0.087378,-0.010660,1.000000,-0.003140
postal_code,0.063686,0.034595,0.015637,-0.015794,-0.004568,-0.003140,1.000000


**Notas:**
- La características númerica que mas afectan el precio del carro son: registration_year (corr = 0.55) y mileage (-0.40). Cuanto mas reciente sea el año de registro mayor sera el precio y cuanto mas alto sea el millaje menor sera el precio. 
- Milege y not_repaired, son las unicas columnas que afectan negativamente.
- La característica númerica que menos afecta positivamente es el mes de registro (corr = 0.008), debido a su baja relación con el precio y las otras varibles, sera descartada. Ademas no aporta un valor adicional al modelo. 
- Se convertira la columna registration_year a los años de antiguedad (año actual - año de registro)

In [21]:
# Eliminar la columna registration_month
car_data_cln.drop('registration_month', axis=1, inplace=True)

In [22]:
# Obtener la antiguedad del auto a partir de la columna de registration_year
# La correlacion pasara de ser positiva a negativa.
año_actual = 2024
car_data_cln['registration_year'] = año_actual - car_data_cln['registration_year'] 
car_data_cln.rename(columns={'registration_year':'years_since_registration'})
car_data_cln

,price,vehicle_type,registration_year,gearbox,power,model,mileage,fuel_type,brand,not_repaired,postal_code
3,1500,small,23,manual,75,golf,150000,petrol,volkswagen,False,91074
4,3600,small,16,manual,69,fabia,90000,gasoline,skoda,False,60437
5,650,sedan,29,manual,102,3er,150000,petrol,bmw,True,33775
6,2200,convertible,20,manual,109,2_reihe,150000,petrol,peugeot,False,67112
10,2000,sedan,20,manual,105,3_reihe,150000,petrol,mazda,False,96224
...,...,...,...,...,...,...,...,...,...,...,...
354359,7900,sedan,14,manual,140,golf,150000,gasoline,volkswagen,False,75223
354360,3999,wagon,19,manual,3,3er,150000,gasoline,bmw,False,81825
354362,3200,sedan,20,manual,225,leon,150000,petrol,seat,True,96465
354366,1199,convertible,24,auto,101,fortwo,125000,petrol,smart,False,26135


In [23]:
# Porcentaje de valores ausentes por columna
car_data_cln.isna().sum()*100/len(car_data_cln)

price                0.0
vehicle_type         0.0
registration_year    0.0
gearbox              0.0
power                0.0
model                0.0
mileage              0.0
fuel_type            0.0
brand                0.0
not_repaired         0.0
postal_code          0.0
dtype: float64

### <a id='toc1_3_11_'></a>[Dividir el dataset en trenamiento y validación](#toc0_)

In [24]:
# Dividir las características del objetivo
features = car_data_cln.drop('price', axis=1)
target = car_data_cln.price

# Dividir el dataset en un conjunto de entrenamiento y validación
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.3, random_state=54321
)

### <a id='toc1_3_12_'></a>[Escalado de datos](#toc0_)

In [25]:
def featureScaler(df, numeric_cols, categ_cols):
    ''' 
    Convierte un dataframe con características númericas
    en un dataframe estandarizado.
    in: dataframe
    out: dataframe estándar para las columnas númericas + categoricas
    '''
    df = df.reset_index(drop=True)
    scaler = StandardScaler()                                                   
    features_scaled = scaler.fit_transform(df[numeric_cols])                            
    df_features_scaled = pd.DataFrame(features_scaled, columns=numeric_cols)   
    df_scaled = pd.concat([df_features_scaled,df[categ_cols]], axis=1)
    return df_scaled

In [26]:
# Definir las columnas numericas y categoricas
feat_num_cols = ['registration_year', 'power', 'mileage', 'postal_code']
feat_categ_cols = ['vehicle_type', 'gearbox', 'model', 'fuel_type', 'brand', 'not_repaired']

# Escalar el conjunto de prueba y entrenamiento 
features_train_scaled = featureScaler(features_train, feat_num_cols, feat_categ_cols)
features_test_scaled = featureScaler(features_test, feat_num_cols, feat_categ_cols)

# Resetear el indice de los objetivos
target_train = target_train.reset_index(drop=True)
target_test = target_test.reset_index(drop=True)

In [27]:
# Crear un dataset con las columnas categoricas en one hot encoding
features_train_scaled_ohe = pd.get_dummies(features_train_scaled, columns=feat_categ_cols, drop_first=True)
features_test_scaled_ohe = pd.get_dummies(features_test_scaled, columns=feat_categ_cols, drop_first=True)

# Crear columnas faltantes en el conjuto de prueba
for col in features_train_scaled_ohe.columns:
    if ~(col in features_test_scaled_ohe):
        features_test_scaled_ohe[col] = 0

# Reordenar las columnas nuevas
features_test_scaled_ohe = features_test_scaled_ohe[features_train_scaled_ohe.columns]

## <a id='toc1_4_'></a>[Entrenamiento del modelo](#toc0_)

### <a id='toc1_4_1_'></a>[Modelo de regresión lineal](#toc0_)

In [28]:
class SGDLinearRegression:
    def __init__(self, step_size=0.001, epochs=100, batch_size=100, reg_weight=10.0):
        self.step_size = step_size
        self.epochs = epochs
        self.batch_size = batch_size
        self.reg_weight = reg_weight

    def fit(self, train_features, train_target):
        X = np.concatenate(
            (np.ones((train_features.shape[0], 1)), train_features), axis=1
        )
        y = train_target
        w = np.zeros(X.shape[1])

        for _ in range(self.epochs):
            batches_count = X.shape[0] // self.batch_size
            for i in range(batches_count):
                begin = i * self.batch_size
                end = (i + 1) * self.batch_size
                X_batch = X[begin:end, :]
                y_batch = y[begin:end]

                gradient = (
                    2
                    * X_batch.T.dot(X_batch.dot(w) - y_batch)
                    / X_batch.shape[0]
                )

                reg = 2 * w.copy()
                reg[0] = 0 
                gradient += self.reg_weight * reg 
                
                w -= self.step_size * gradient

        self.w = w[1:]
        self.w0 = w[0]

    def predict(self, test_features):
        return test_features.dot(self.w) + self.w0

In [29]:
# Medir el tiempo 

# Los mejores hiperparámetros encontrados par el modelo de regresión lineal
# Se utilizara como prueba de cordura
model = SGDLinearRegression(0.01, 10, 100, 1.0)

# Convertir los valores 
%time model.fit(features_train_scaled_ohe.astype(float), target_train)
pred_train = model.predict(features_train_scaled_ohe.astype(float))
pred_test = model.predict(features_test_scaled_ohe.astype(float))
rmse_train = mean_squared_error(target_train, pred_train) ** 0.5
rmse_test = mean_squared_error(target_test, pred_test) ** 0.5
print(f'RMSE Train: {rmse_train}')
print(f'RMSE Test: {rmse_test}')

CPU times: user 12.2 s, sys: 21.3 s, total: 33.5 s
Wall time: 33.5 s
RMSE Train: 3703.074237852838
RMSE Test: 4753.702665356179


### <a id='toc1_4_2_'></a>[Modelo de bosque aleatorio basico](#toc0_)

In [30]:
# Ya se incluyen los mejores hiperaparametros
gb_model = GradientBoostingRegressor(
    learning_rate=0.0045, max_depth=23, min_samples_leaf=7, min_samples_split=2, n_estimators=5, random_state=54321
)
%time gb_model.fit(features_train_scaled_ohe.astype(float), target_train)
predicts = gb_model.predict(features_test_scaled_ohe.astype(float))
mse = mean_squared_error(target_test, predicts) ** 0.5
print(f'Mean Squared Error en conjunto de prueba: {mse}')

CPU times: user 20.7 s, sys: 167 ms, total: 20.9 s
Wall time: 20.9 s
Mean Squared Error en conjunto de prueba: 4730.943342117087


**Nota:** No hubo una mejora significativa en el error, solo hay una diferencia de 23 USD de error con respecto al modelo de regresión lineal. En cuestión del tiempo hubo una reducción del 39.22%.

### <a id='toc1_4_3_'></a>[Catboost](#toc0_)

In [31]:
model = CatBoostRegressor(iterations=1000, depth=10, learning_rate=0.1, loss_function='RMSE')
%time model.fit(features_train, target_train, cat_features=feat_categ_cols)
predicts = model.predict(features_test)
rmse = mean_squared_error(target_test, predicts) ** 0.5
print(f'RMSE: {rmse}')

0:	learn: 4390.3871574	total: 615ms	remaining: 10m 14s
1:	learn: 4074.2645037	total: 1.17s	remaining: 9m 41s
2:	learn: 3794.3828704	total: 1.69s	remaining: 9m 22s
3:	learn: 3548.4626571	total: 2.24s	remaining: 9m 18s
4:	learn: 3328.6763183	total: 2.7s	remaining: 8m 58s
5:	learn: 3138.4907059	total: 3.2s	remaining: 8m 50s
6:	learn: 2972.7036806	total: 3.69s	remaining: 8m 43s
7:	learn: 2819.7947326	total: 4.18s	remaining: 8m 38s
8:	learn: 2683.8801755	total: 4.68s	remaining: 8m 35s
9:	learn: 2565.0726536	total: 5.18s	remaining: 8m 32s
10:	learn: 2465.5441394	total: 5.7s	remaining: 8m 32s
11:	learn: 2374.3299799	total: 6.16s	remaining: 8m 27s
12:	learn: 2299.3358678	total: 6.68s	remaining: 8m 27s
13:	learn: 2231.1783489	total: 7.09s	remaining: 8m 19s
14:	learn: 2173.4459022	total: 7.53s	remaining: 8m 14s
15:	learn: 2121.8912067	total: 8.06s	remaining: 8m 15s
16:	learn: 2077.7091089	total: 8.58s	remaining: 8m 16s
17:	learn: 2039.9099637	total: 9.01s	remaining: 8m 11s
18:	learn: 2003.370275

In [32]:
# Implementacion con LightGBM

# Crear un conjunto de lgbm de entrenamiento
features_train['brand'] = features_train['brand'].astype('category').cat.codes
features_train['model'] = features_train['model'].astype('category').cat.codes

train_data = lgb.Dataset(features_train, label=target_train)

# Crear un conjunto de lgbm de prueba
features_test['brand'] = features_test['brand'].astype('category').cat.codes
features_test['model'] = features_test['model'].astype('category').cat.codes



# Definir los parámetros del modelo
# Es un modelo de regresión, se evaluara con la raíz del error cuadrático medio
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'boosting_type': 'gbdt',
    'num_leaves': 41,
    'learning_rate': 0.1,
    'feature_fraction': 0.9
}

# Entrenar el modelo
num_round = 1000
%time bst = lgb.train(params, train_data, num_round)

# Realiza predicciones en el conjunto de prueba
predicts = bst.predict(features_test, num_iteration=bst.best_iteration)

# Evaluar con RMSE
rmse = mean_squared_error(target_test, predicts) ** 0.5
print(f'RMSE: {rmse}')

/tmp/ipykernel_91/3649278590.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_train['brand'] = features_train['brand'].astype('category').cat.codes
/tmp/ipykernel_91/3649278590.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_train['model'] = features_train['model'].astype('category').cat.codes
/tmp/ipykernel_91/3649278590.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004627 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 899
[LightGBM] [Info] Number of data points in the train set: 147571, number of used features: 10
[LightGBM] [Info] Start training from score 5354.426195
CPU times: user 16.6 s, sys: 126 ms, total: 16.8 s
Wall time: 16.9 s
RMSE: 1624.2413826017148


**Nota:** Hubo una mejora significativa en tiempo comparación de la regresión lineal, quedando a la par con el bosque aleatorio basico. Sin embargo la mejora real consiste en el RMSE que es comparable con Catboost pero mucho mas rápido. Probablemente el mejor modelo hasta ahora.

### <a id='toc1_4_4_'></a>[XGBoost](#toc0_)

In [35]:
# Implementacion con XGBoost
# Crear una matriz DMatrix para XGBoost
dtrain = xgb.DMatrix(features_train_scaled_ohe, label=target_train)
dtest = xgb.DMatrix(features_test_scaled_ohe, label=target_test)

# Definir los parámetros del modelo
# Regresión y se evalua con la raíz del error cuadrático medio
params = {
    'objective': 'reg:squarederror',  
    'eval_metric': 'rmse',
    'max_depth': 34,
    'learning_rate': 0.1,
    'subsample': 0.8
}

# Entrenar el modelo
num_round = 1
%time bst = xgb.train(params, dtrain, num_round)

# Realizar predicciones en el conjunto de prueba
predicts = bst.predict(dtest)

# Evalúa el rendimiento del modelo con RMSE
rmse = mean_squared_error(target_test, predicts) ** 0.5
print(f'RMSE: {rmse}')

CPU times: user 30.5 s, sys: 260 ms, total: 30.8 s
Wall time: 30.9 s
RMSE: 6932.95591701738


**Nota:**
Para este modelo se necesita one hot encoding. Quiza por eso es mucho mas lento que los otros. Por esta razon los hiperparametros no se cambiaron mucho. 

Se hizo una prueba en visual studio con num_round = 100. Y los resultados en tiempo fueron 481 s de entrenamiento y 4845.69 RMSE. Este modelo tuvo peor desempeño que la prueba de cordura. Los hiperparametros podrian mejorarse pero el tiempo de entrenamiento es muy largo.

## <a id='toc1_5_'></a>[Análisis del modelo](#toc0_)

La siguiente tabla muestra la comparación del tiempo de entrenamiento y una calidad que se obtuvo a partir de la raíz del error cuadrático medio (RMSE) de distintos modelos con los mejores hiperprámetros. La prueba de cordura se definio con el modelo del regresión lineal. Se destaca el modelo LightGBM que obtuvo un tiempo de entrenamiento relativamente bajo y obtuvo el segundo mejor puedo en calidad. Sin embargo Catboost obtuvo un error menor, el timpo fue de 9 mins aproxidamente, creo que es un tiempo tolerable de entrenamiento. 
  
|          Modelo          | Wall Time de entrenamiento | Calidad (RMSE) | Diferencia en tiempo | Diferencia en calidad |
|--------------------------|----------------------------|----------------|----------------------|-----------------------|
| Regresión Lineal         |            35.4s           |     4753.70    |        N/A           |         N/A           |
| Bosque aleatorio (basico)|            21.5s           |     4730.94    |      -39.22%         |        -0.10%         |
|        Catboost          |           522.0s           |     1542.58    |    +1378.53%         |       -67.59%         |
|        LightGBM          |            20.1s           |     1624.24    |      -43.22%         |       -66.00%         |
|        XGBoost           |           481.0s           |     4845.69    |    +1257.63%         |       +1.94%          |